# The project of ecsploration of data of depression

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [7]:
raw_data_df = pd.read_csv('./data/27521-0001-Data.tsv', sep='\t')
raw_data_df.head()

,CASEID,QUESTID,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,LARCENY,MTRTHEFT,FRAUD,PROPCRME,DRUG,DRUGSALE,ALLARRST,SAXMDE,MDEGT12M,SAXMDEGT
0,1,2000033,1,99,99,18,9999,99,1,3,...,0,0,0,0,0,0,0,0,0,0
1,2,2000140,1,99,99,13,9999,99,4,93,...,0,0,0,0,0,0,0,0,0,0
2,3,2000202,1,99,99,28,9999,99,4,93,...,0,0,0,0,0,0,0,0,0,0
3,4,2000220,1,99,99,12,9999,99,4,93,...,0,0,0,0,0,0,0,0,0,0
4,5,2000257,1,99,99,12,9999,99,4,93,...,0,0,0,0,0,0,0,0,0,0


In [19]:
columns = ["DEPRESSIONINDEX", "ANYTXRXMDE", "CATAG7", "IRSEX", "NEWRACE2",
           "ANYINDEX", "MJANDCOKE", "ILLICITDRUGUSE", "LSYRILLICIT", "COKECRACK", "OTHERILLICIT", "MARJLTYR",
           "MJCOKELY", "COCCRKLY", "ALCFMFPB", "IREDUC2", "INCOME", "INCOME_R", "IRPRVHLT", "WORKFORCE", 
           "EMPSTAT4", "REVERSEPOP", "MOVESPY2", "CACHAR", "CATYPE", "CRIMEHIST", "ANYSDRUG", "ANYATTACK", 
           "ANYTHEFT", "NUMARREST", "HEALTH2", "DSTNCALM", "DSTTIRE", "DSTCHEER", "IRMARIT", "RKIDSHH", 
           "CHILDRENINHOME"]
raw_data_df = raw_data_df[columns]
print(raw_data_df.shape)
raw_data_df.head()

(55602, 37)


,DEPRESSIONINDEX,ANYTXRXMDE,CATAG7,IRSEX,NEWRACE2,ANYINDEX,MJANDCOKE,ILLICITDRUGUSE,LSYRILLICIT,COKECRACK,...,ANYATTACK,ANYTHEFT,NUMARREST,HEALTH2,DSTNCALM,DSTTIRE,DSTCHEER,IRMARIT,RKIDSHH,CHILDRENINHOME
0,0,0,5,1,7,0.142857,1,1,1,1,...,0,0,0,1,99,99,5,4,0,0
1,0,0,4,1,1,0.142857,1,1,0,0,...,0,0,0,2,99,99,4,4,0,0
2,0,0,7,1,1,0.142857,0,0,0,0,...,0,0,0,1,99,99,5,1,0,0
3,0,0,5,1,1,0.500000,1,1,1,0,...,0,0,0,2,99,99,5,4,0,0
4,0,0,7,2,1,0.142857,1,1,0,0,...,0,0,0,2,99,99,4,1,1,1


clad = complicated, look at documentation
brak odpowiedzi     = missing values = -9
w przypadku float   - missing vlues = -1
legitimate skip     - 99


DEPRESSIONINDEX (0-9)       - natężenie depresji 
ANYTXRXMDE      binary      - leki w zeszlym roku  1 - yes
CATAG7          (1-7)       - 1 - najmlodsza grupa wiekowa
IRSEX           binary      - 1 - facet, 2 - kobieta
NEWRACE2        (1-7)       - 1 - white, 2 - black/AfrAm
ANYINDEX        float 0-1   - 1 - wysoka zależność - wskaźnik uzależnienia
MJANDCOKE	    binary      - drug
ILLICITDRUGUSE	binary      - drug
LSYRILLICIT	    binary      - drug
COKECRACK       binary      - drug
OTHERILLICIT    binary      - drug
MARJLTYR        binary      - drug to be changed MARLY
MJCOKELY        binary      - drug 
COCCRKLY        binary      - drug
ALCFMFPB        clad        - alcohol last year
IREDUC2         (1-11)      - w której klasie osoba porzuciła naukę
INCOME          (1-4)       - ile rodzina zarabia, widełki
INCOME_R        (1-7)       - ile dana osoba zarabia, widełki
IRPRVHLT        (1-2)       - 1 - ma prywatne ubezpieczenie zdrowotne, to be changed to binary
WORKFORCE       binary      - 1 - working, 0 - not
EMPSTAT4        (1-4),99    - 1 - pracuje full-time, 2 - part time, 99 - młodociani
REVERSEPOP      (1-3)       - 1 - nie jest metropolią, 2 - jest, 3 - ogromna metropolia
MOVESPY2        clad        - przeprowadzka w zeszlym roku
CACHAR          (1-9)       - gdzie mieszka
CATYPE          (1-6)       - jakie zabezpieczenia do mieszkania
CRIMEHIST       binary      - 0 - brak historii kriminalnej
ANYSDRUG        binary      - 0 - nie sprzedawał drugów LY
ANYATTACK       binary      - 0 - nie atakował w zeszł ym roku, to be changed ANYATTACKLY
ANYTHEFT        binary      - 0 - brak kradzieży LY
NUMARREST       (0-3)       - ile razy aresztowany w zeszłym roku
HEALTH2         (1-4)       - 1 - swietne zdrowie
DSTNCALM        clad        - jak czesto nie mógł sie uspokoić w najgorszych miesiącach
DSTTIRE         clad        - jak czesto był wyczerpany
DSTCHEER        clad        - jak czesto nie mogl sie pocieszyc w najgorszych miesiacach
IRMARIT         (1-4)99     - 1 - married, 99 - za młody
RKIDSHH         binary      - dzieci ponizej 18 lat w liceum, 0-brak dzieci
CHILDRENINHOME  binary      - 0 - brak dzieci w domu, 1 - są dzieci w domu


jeśli zmienna w dokumentacji miała ponad 70% odpowiedzi legitimate skip - to usuwać, brak dobrych odpowiedzi
najciekawsze pytania, typu problemy ze snem w najgorszych miesiącach, albo o mysli samobójcze - mają po 95% legitimate skip! nie brałem ich pod uwagę, nic nie wnoszą, szkoda że badani nie udzielili odpowiedzi.
wyrzucałem dane, które w mojej opinii przekazywały te samą informację, która już była zawarta w innych kolumnach
jak było mało wierszy, które miały w danej zmiennej -1 lub -9 to usuwałem wiersz. 
na pewno narkotyki będą miały podobne korelacje, nie wiem czy tylko nie zostawić czy brał narkotyki czy nie, ale na razie zostawie.